In [1]:
import time
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
def create_dir(dir):
    """
    Checks if directory exists and if not it will create it
    
    dir: str
        full path the directory
    """
    if not os.path.exists(dir):
            os.mkdir(dir)

In [6]:
# Pick STAR, LOCA2, or LOCA2STAR
model_set = 'LOCA2STAR'

MODELS_TO_PROCESS = ['ACCESS-CM2',
                     'ACCESS-ESM1-5',
                     'BCC-CSM2-MR',
                     'CanESM5',
                     'EC-Earth3',
                     'FGOALS-g3',
                     'GFDL-ESM4',
                     'INM-CM4-8',
                     'INM-CM5-0',
                     'IPSL-CM6A-LR',
                     'MIROC6',
                     'MPI-ESM1-2-HR',
                     'MPI-ESM1-2-LR',
                     'MRI-ESM2-0',
                     'NorESM2-LM',
                     'NorESM2-MM'
                    ]

# pick variables
VARIABLES_TO_PROCESS = ['cdd']

SSPS_STAR_TO_PROCESS = ['ssp245', 'ssp585']
SSPS_LOCA2_TO_PROCESS = ['ssp245', 'ssp370', 'ssp585']
SSPS_LOCA2STAR_TO_PROCESS = ['ssp245', 'ssp585']

ENSEMBLE_WEIGHTS = {
                    "ACCESS-CM2" : 0.0412,
                    "ACCESS-ESM1-5" : 0.0581,
                    "BCC-CSM2-MR" : 0.0723,
                    "CanESM5" : 0.029,
                    "EC-Earth3" : 0.0498,
                    "FGOALS-g3" : 0.0716,
                    "GFDL-ESM4" : 0.0589,
                    "INM-CM4-8" : 0.0646,
                    "INM-CM5-0" : 0.0649,
                    "IPSL-CM6A-LR" : 0.0449,
                    "MIROC6" : 0.0767,
                    "MPI-ESM1-2-HR" : 0.0731,
                    "MPI-ESM1-2-LR" : 0.0755,
                    "MRI-ESM2-0" : 0.073,
                    "NorESM2-LM" : 0.0736,
                    "NorESM2-MM" : 0.0727
                    }

In [7]:
input_dir = r'C:\Users\raf14049\OneDrive - Esri\Documents\CRIS\data\resample_mask'
output_dir = r'C:\Users\raf14049\OneDrive - Esri\Documents\CRIS\data\ensemble'

create_dir(output_dir)

In [8]:
# for model_set in MODEL_SET_TO_PROCESS:

if model_set == 'STAR' or model_set == 'LOCA2':

    for variable in VARIABLES_TO_PROCESS:
        
        if model_set == 'STAR':
            SSPS_TO_PROCESS = SSPS_STAR_TO_PROCESS
        elif model_set == 'LOCA2':
            SSPS_TO_PROCESS = SSPS_LOCA2_TO_PROCESS

        for ssp in SSPS_TO_PROCESS:

            start_proc_tm = time.time()

            ds = xr.open_dataset(os.path.join(input_dir, model_set, variable, model_set + '_' + MODELS_TO_PROCESS[0] + '_' + variable + '_' + ssp + '_1950_2100.nc'))
            ds[variable.replace('-','_')].values *= ENSEMBLE_WEIGHTS.get(MODELS_TO_PROCESS[0])

            for model in MODELS_TO_PROCESS[1:]:

                ds_temp = xr.open_dataset(os.path.join(input_dir, model_set, variable, model_set + '_' + model + '_' + variable + '_' + ssp + '_1950_2100.nc'))
                ds_temp[variable.replace('-','_')].values *= ENSEMBLE_WEIGHTS.get(model)
                ds[variable.replace('-','_')].values += ds_temp[variable.replace('-','_')].values

            out_file = os.path.join(output_dir, model_set, model_set + '_' + variable + '_' + ssp + '_1950_2100.nc')

            create_dir(os.path.join(output_dir, model_set))
            
            ds.to_netcdf(out_file, encoding={variable.replace('-','_'): {'zlib': True, 'complevel': 9}})
            
            end_proc_tm = time.time()
            elapsed_proc_tm = end_proc_tm - start_proc_tm
            print(f'Successfully processed: {out_file}')
            print(f"[Processed time: {elapsed_proc_tm/60:0,.2f} minutes]")

elif model_set == 'LOCA2STAR':

    for variable in VARIABLES_TO_PROCESS:
    
        for ssp in SSPS_LOCA2STAR_TO_PROCESS:

            start_proc_tm = time.time()

            ds_star = xr.open_dataset(os.path.join(input_dir, 'STAR', variable, 'STAR_' + MODELS_TO_PROCESS[0] + '_' + variable + '_' + ssp + '_1950_2100.nc'))
            ds_star[variable.replace('-','_')].values *= ENSEMBLE_WEIGHTS.get(MODELS_TO_PROCESS[0])

            ds_loca2 = xr.open_dataset(os.path.join(input_dir, 'LOCA2', variable, 'LOCA2_' + MODELS_TO_PROCESS[0] + '_' + variable + '_' + ssp + '_1950_2100.nc'))
            ds_loca2[variable.replace('-','_')].values *= ENSEMBLE_WEIGHTS.get(MODELS_TO_PROCESS[0])

            for model in MODELS_TO_PROCESS[1:]:

                ds_star_temp = xr.open_dataset(os.path.join(input_dir, 'STAR', variable, 'STAR_' + model + '_' + variable + '_' + ssp + '_1950_2100.nc'))
                ds_star_temp[variable.replace('-','_')].values *= ENSEMBLE_WEIGHTS.get(model)
                ds_star[variable.replace('-','_')].values += ds_star_temp[variable.replace('-','_')].values

                ds_loca2_temp = xr.open_dataset(os.path.join(input_dir, 'LOCA2', variable, 'LOCA2_' + model + '_' + variable + '_' + ssp + '_1950_2100.nc'))
                ds_loca2_temp[variable.replace('-','_')].values *= ENSEMBLE_WEIGHTS.get(model)
                ds_loca2[variable.replace('-','_')].values += ds_loca2_temp[variable.replace('-','_')].values

            out_file = os.path.join(output_dir, model_set, model_set + '_' + variable + '_' + ssp + '_1950_2100.nc')

            create_dir(os.path.join(output_dir, model_set))
            
            ds_star[variable.replace('-','_')].values += ds_loca2[variable.replace('-','_')].values
            ds_ensemble = ds_star
            ds_ensemble[variable.replace('-','_')].values /= 2
            
            ds_ensemble.to_netcdf(out_file, encoding={variable.replace('-','_'): {'zlib': True, 'complevel': 9}})
            
            end_proc_tm = time.time()
            elapsed_proc_tm = end_proc_tm - start_proc_tm
            print(f'Successfully processed: {out_file}')
            print(f"[Processed time: {elapsed_proc_tm/60:0,.2f} minutes]")

Successfully processed: C:\Users\raf14049\OneDrive - Esri\Documents\CRIS\data\ensemble\LOCA2STAR\LOCA2STAR_cdd_ssp245_1950_2100.nc
[Processed time: 0.27 minutes]
Successfully processed: C:\Users\raf14049\OneDrive - Esri\Documents\CRIS\data\ensemble\LOCA2STAR\LOCA2STAR_cdd_ssp585_1950_2100.nc
[Processed time: 0.26 minutes]
